<a href="https://colab.research.google.com/github/gdabrow/Investments/blob/main/live_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook is about real-time cryptocurrency trading on the Binance exchange**

In [ ]:
!pip install pandas_ta
!pip install python-binance

In [ ]:
from binance.client import Client
import pandas as pd
import pandas_ta as ta
import time

client = Client()

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
cryptocurrency = 'BTCUSDT'

#download the latest data from Binance
dane=pd.DataFrame(client.get_historical_klines(cryptocurrency, '1h','2h ago UTC' ), dtype=float)

#finding last price
close=dane[[4]]
last_price = close[-1] 

#setting initial variables

#are we out of the market ?
traid = 'out'
#stop loss for buy
slb=0
#iterator
i=0

interval='1h'
backTo='35h ago UTC'

#length of the moving average
#https://www.investopedia.com/terms/e/ema.asp
ema=30

while True:

  try:
    #download data from Binance
    dane=pd.DataFrame(client.get_historical_klines(cryptocurrency, interval, backTo), dtype=float)
  except :
    time.sleep(2)
    continue

  dane=dane[[0,1,2,3,4]]
  dane.columns=['Date','Open','High','Low','Close']

  #calculation of data for a moving average
  dane['EMA']=ta.ema(close=dane.Close, length=ema).iloc[:,]

  #if slb != 0 that means I'm in the market and the price hasn't increased by 5% yet
  if slb != 0:
    #checking if the price has increased above 5% since the purchase
    if dane.Close.iloc[-1] < price_buy * 1.005:
      # checking if the price fell below slb and EMA at the same time
      if dane.Close.iloc[-1] < slb and dane.Close.iloc[-1] < dane.EMA.iloc[-1]:
        #if the above conditions are met, we exit the market
        traid = 'out'
        slb = 0
        price_sell = dane.Close.iloc[-1]
        #throw the info to the consoles
        print(str(price_sell)+' sell from slb')

    #we are already 5% above the purchase price and we no longer protect ourselves
    else:
      slb = 0

  #check if we are outside the market and if the price has crossed the ema from the bottom
  elif (dane.Close.iloc[-1] > dane.EMA.iloc[-1]) and (last_price < dane.EMA.iloc[-1]) and traid == 'out':
    #if so, we make a purchase
    traid = 'buy'
    #note the purchase price
    price_buy = dane.Close.iloc[-1]
    #set slb to 95% of the purchase price
    slb = dane.EMA.iloc[-1]*0.995
    #throw the info to the consoles
    print(str(price_buy)+' buy from clasic')

  #check if we are in the market and if the price has crossed the ema from the top
  elif (dane.Close.iloc[-1] < dane.EMA.iloc[-1]) and (last_price > dane.EMA.iloc[-1]) and traid == 'buy':
    #if so, we close the position
    traid = 'out'
    #set slb to 0 of the purchase price
    slb = 0
    #note the sell price
    price_sell = dane.Close.iloc[-1]
    #throw the info to the consoles
    print(str(price_sell)+' sell from clasic')


  i+=1
  #cyclic tracking of parameters on the console
  if i == 50:
    print(dane.Close.iloc[-1], dane.EMA.iloc[-1])
    i=0

  time.sleep(1)